# Tracking Identity Consistency

This notebook shows how to aggregate IDF1 metrics for multiple trackers/backbones using the `tools/eval_reid.py` script.

In [ ]:
import json
import pathlib
import pandas as pd
import subprocess

PROJECT_ROOT = pathlib.Path('..').resolve()
EVAL_SCRIPT = PROJECT_ROOT / 'tools' / 'eval_reid.py'
GT_FILE = PROJECT_ROOT / 'datasets' / 'mot' / 'train' / 'MOT17-02' / 'gt' / 'gt.txt'
RUNS = {
    'osnet': PROJECT_ROOT / 'artifacts' / 'tracks' / 'osnet_MOT17-02.txt',
    'dinov2_vitl14': PROJECT_ROOT / 'artifacts' / 'tracks' / 'dinov2_vitl14_MOT17-02.txt',
}


In [ ]:
records = []
for name, track_path in RUNS.items():
    if not track_path.exists():
        print(f'Skipping {name}: missing track file {track_path}')
        continue
    completed = subprocess.run(
        ['python', str(EVAL_SCRIPT), 'idf1', '--gt', str(GT_FILE), '--pred', str(track_path)],
        capture_output=True,
        check=True,
        text=True,
    )
    metrics = json.loads(completed.stdout)
    records.append({
        'model': name,
        'IDF1': metrics['IDF1'],
        'IDP': metrics['IDP'],
        'IDR': metrics['IDR'],
    })

idf1_table = pd.DataFrame.from_records(records)
idf1_table.sort_values('IDF1', ascending=False, inplace=True)
idf1_table.reset_index(drop=True, inplace=True)
idf1_table